<a href="https://colab.research.google.com/github/PhillipMas/PhillipMas/blob/main/Parameter_Tuning_Optimization_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [3]:
# Fetch data
ticker = '^J203.JO'  # Example ticker
data = yf.download(ticker, start='2012-01-01', end='2024-07-21')
data['Log Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
data = data.dropna()  # Drop NaN values

# Prepare data for LSTM
window_size = 120
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data['Log Returns'].values.reshape(-1, 1))

# Function to create rolling windows
def create_rolling_windows(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

# Create rolling windows
X, y = create_rolling_windows(scaled_data, window_size)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

[*********************100%***********************]  1 of 1 completed


In [5]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00


In [ ]:
import optuna
from keras.layers import Dropout
from keras.optimizers import Adam

# Define the objective function
def objective(trial):
    # Suggest hyperparameters
    units = trial.suggest_int('units', 50, 200, step=50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5, step=0.1)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_int('batch_size', 16, 128, step=16)
    epochs = trial.suggest_int('epochs', 50, 200, step=50)

    # Build the model
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluate the model
    loss = model.evaluate(X, y, verbose=0)
    return loss

# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters: ", best_params)

# Build and train the model with the best hyperparameters
model = Sequential()
model.add(LSTM(units=best_params['units'], return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(best_params['dropout_rate']))
model.add(LSTM(units=best_params['units']))
model.add(Dropout(best_params['dropout_rate']))
model.add(Dense(1))

optimizer = Adam(learning_rate=best_params['learning_rate'])
model.compile(optimizer=optimizer, loss='mean_squared_error')

model.fit(X, y, epochs=best_params['epochs'], batch_size=best_params['batch_size'])


[I 2024-09-11 08:21:25,305] A new study created in memory with name: no-name-bcca220f-c6ff-4c0e-960c-0af772945aa4
<ipython-input-6-4bd0a84711e7>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[I 2024-09-11 09:09:56,908] Trial 0 finished with value: 0.11581242084503174 and parameters: {'units': 100, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.0017703520437827279, 'batch_size': 96, 'epochs': 150}. Best is trial 0 with value: 0.11581242084503174.
